In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import networkx as nx
import scipy
import pickle
%matplotlib inline
%load_ext autoreload
%autoreload 2


In [1]:
import irlb

ModuleNotFoundError: No module named 'irlb'

In [2]:
import irlbpy

ModuleNotFoundError: No module named 'irlbpy'

In [3]:
!pip install -e git+https://github.com/bwlewis/irlbpy.git#egg=irlbpy

Obtaining irlbpy from git+https://github.com/bwlewis/irlbpy.git#egg=irlbpy
  Cloning https://github.com/bwlewis/irlbpy.git to ./src/irlbpy
  Running command git clone --filter=blob:none --quiet https://github.com/bwlewis/irlbpy.git /home/akhil/cellag-misc/src/irlbpy
  Resolved https://github.com/bwlewis/irlbpy.git to commit efe58d2f60f8fe4f4b7be40c1ca2ad4392129aca
  Preparing metadata (setup.py) ... done
  Running setup.py develop for irlbpy


In [8]:
import irlb

ModuleNotFoundError: No module named 'irlb'

In [9]:
import irlbpy

ModuleNotFoundError: No module named 'irlbpy'

In [10]:
import importlib  
foobar = importlib.import_module("irlbpy-0.1.0")

ModuleNotFoundError: No module named 'irlbpy-0'